### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.

In [ ]:
# type: ignore

import os
import json
import datetime

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
index = ''  # Set a matching value here and in model-routinbg.ipynb if you want separate instances of the lab. This is helpful when tearing down resources and mitigating API Management's soft delete.
resource_group_name = f"lab-{deployment_name}{index}"

def log(stdout, name, action):
    if stdout.n.startswith("ERROR"):
        print("👎🏻 ", name, " was NOT ", action, ": ", stdout)
    else:
        print("👍🏻 ", name, " was ", action, " ⌚ ", datetime.datetime.now().time())

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} -o json
deployment = json.loads(deployment_stdout.n)

for resource in deployment.get("properties").get("outputResources"):
    resource_id = resource.get("id")
    try:
        query = "\"{type:type, name:name, location:location}\""
        resource_stdout = ! az resource show --id {resource_id} --query {query} -o json
        resource = json.loads(resource_stdout.n)
        resource_name = resource.get("name")
        resource_type = resource.get("type")
        if resource_type == "Microsoft.CognitiveServices/accounts":
            resource_location = "\"" + resource.get("location") + "\""
            delete_stdout = ! az cognitiveservices account delete -g {resource_group_name} -n {resource_name}
            log(delete_stdout, resource_name, "deleted")
            delete_stdout = ! az cognitiveservices account purge -g {resource_group_name} -n {resource_name} -l {resource_location}
            log(delete_stdout, resource_name, "purged")
        elif resource_type == "Microsoft.ApiManagement/service":
            resource_location = "\"" + resource.get("location") + "\""
            delete_stdout = ! az apim delete -n {resource_name} -g {resource_group_name} -y
            log(delete_stdout, resource_name, "deleted")
            delete_stdout = ! az apim deletedservice purge --service-name {resource_name} --location {resource_location}
            log(delete_stdout, resource_name, "purged")
    except:
        print("✌🏻 ", resource_id, " ignored")

delete_stdout = ! az group delete --name {resource_group_name} -y
log(delete_stdout, resource_group_name, "deleted")

